In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
## Simulate Data

n_obs = 2000
n_cols = 50

X = pd.DataFrame(np.random.rand(n_obs, n_cols), columns=range(n_cols))
coefs = np.random.rand(n_cols)
coefs[10] = -10

probs = 1 / (1 + np.exp(-(-5 + (coefs*X).sum(axis=1) + 0.1*np.random.normal(size=n_obs))))
y = pd.Series([0]*len(probs))
y[probs>0.5] = 1
print("Percent y=1:\t" + str(np.sum(y) / float(len(y))))
print("Percent y=0:\t" + str(1 - (np.sum(y) / float(len(y)))))

Percent y=1:	0.6805
Percent y=0:	0.3195


In [3]:
## Random Forest

%time rf_model = RandomForestClassifier(n_estimators=500, n_jobs=12)
%time rf_model.fit(X, y)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 56 µs
CPU times: user 5.22 s, sys: 340 ms, total: 5.56 s
Wall time: 1.66 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=12, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [4]:
def global_marginal_effects(rf_model, X, epsilon_percent=1.0):
    # initialize output
    dydx = pd.DataFrame(np.nan, index=X.index, columns=X.columns)
    
    data_range = X.max() - X.min()
    categorical_boolean = (data_range==1)
    epsilon = (epsilon_percent / 2 / 100) * data_range
    
    # populate dydx
    for i, col in enumerate(X.columns):
        # Store old copy of X[col]
        X_old = X[col].copy()
        
        # Store max and min
        Xmax = X[col].max()
        Xmin = X[col].min()
        
        # If categorical set X_hi to 1
        if((Xmax == 1) & (Xmin == 0)): X[col] = 1
        else: X[col] = X[col] + epsilon[col]
        y_hi = rf_model.predict_proba(X)[:,1]
        
        # If categorical set X_lo to 0
        if((Xmax == 1) & (Xmin == 0)): X[col] = 0
        else: X[col] = X[col] - epsilon[col]
        y_lo = rf_model.predict_proba(X)[:,1]

        # Calculate dydx at each point for column
        dydx.loc[:, col] = y_hi - y_lo
        
        # Reset X[col]
        X[col] = X_old
    
    ## Original interpretation is epsilon % of range, new interpretation is 95% of range
    dydx.loc[:,~categorical_boolean] = dydx.loc[:,~categorical_boolean] * 95 / epsilon_percent
    
    return dydx, dydx.mean()

In [20]:
## add epsilon as kwarg
## add Xmax and Xmin as kwarg
def marginal_effects_column(Xcol, rf_model, X, epsilon_percent=1.0):
    # Store max and min
    Xmax = Xcol.max()
    Xmin = Xcol.min()
    epsilon = (epsilon_percent / 2 / 100) * (Xmax-Xmin)
    
    # Check if categorical
    if((Xmax == 1) & (Xmin == 0)): is_categorical = True
    else: is_categorical=False
    
    if(is_categorical): Xcol = 1
    else: Xcol = Xcol + epsilon
    X[Xcol.name] = Xcol
    y_hi = rf_model.predict_proba(X)[:,1]

    if(is_categorical): Xcol = 0
    else: Xcol = Xcol - epsilon
    X[Xcol.name] = Xcol
    y_lo = rf_model.predict_proba(X)[:,1]

    # Calculate dydx at each point for column
    dydx = y_hi - y_lo
    
    ## Original interpretation is epsilon % of range, new interpretation is 95% of range
    if(not is_categorical):
        dydx = dydx * 95 / epsilon_percent
    
    return dydx

In [21]:
from multiprocessing import Pool
pool = Pool(5)

In [23]:
%%time
results = [pool.apply_async(marginal_effects_column, args=(X[x], rf_model, X)) for x in X]
output = [p.get() for p in results]

CPU times: user 1.62 s, sys: 740 ms, total: 2.36 s
Wall time: 20.9 s


In [24]:
%time dydx, mean = global_marginal_effects(rf_model, X)

CPU times: user 48.3 s, sys: 15.6 s, total: 1min 3s
Wall time: 56.1 s


In [25]:
dydx.shape

(2000, 50)

In [26]:
dydx2 = pd.DataFrame(output).transpose()

In [7]:
dydx.mean()

0    -0.011875
1     0.012065
2     0.020330
3     0.027455
4     0.019190
5     0.002090
6    -0.005510
7    -0.003420
8    -0.006175
9     0.018430
10   -0.376485
11    0.041040
12   -0.012350
13    0.003515
14    0.018145
15    0.002565
16    0.030020
17    0.002755
18    0.015770
19    0.026315
20    0.006935
21    0.022990
22   -0.004940
23    0.005415
24    0.006270
25   -0.018905
26    0.036195
27    0.005985
28    0.006175
29    0.001710
30    0.000665
31    0.000285
32   -0.011400
33    0.010165
34    0.031160
35    0.025745
36    0.013680
37    0.018145
38    0.026410
39    0.042370
40    0.025175
41    0.004560
42   -0.001520
43    0.006460
44    0.019000
45    0.000665
46    0.002185
47   -0.012065
48    0.026695
49    0.017575
dtype: float64

In [ ]:
# 4500 features ~ 8 hours
# Seems O(n)
# 100 features - 638 secs
# 30 features - 200 secs
# 3 features - 11